# Gleam
This notebook aims to design, train, validate, and use a convolutional neural network that is able to predict the distribution of population of a geographical area from nighttime satellite imagery. The product is a high resolution approximation map of .

# Datasets

[Recommended source nighttime pictures (NOAA)](https://ngdc.noaa.gov/eog/viirs/download_dnb_composites.html)

[Recommended source population dataset (SEDAC)](http://sedac.ciesin.columbia.edu/data/set/gpw-v4-population-count-adjusted-to-2015-unwpp-country-totals-rev10)

This data needs to be processed with a GIS tool to meet a few requirements before training :
- Rasters of the same year need to be merged in one GeoTIFF file : lights on the first band, population on the second band.
- Since both sets don't have the same resolution, upscaling the population set might have increased population density. This needs to be adjusted before training. In the case of the recommended datasets, every population pixel needs to be devided by 4 after merging with gdal_merge or QGIS.
- Use a vector file to clip the raster and isolate the region of interest ([recommended vector file to clip by countries (Natural Earth)](https://www.naturalearthdata.com/downloads/10m-cultural-vectors/10m-admin-0-countries/)). Areas with a population of zero (out of borders or in the ocean) will be ignored during preprocessing.

## Area of interest
The population distribution we want to predict is the on living in Colombia. The data we have on Colombia is outdated and has a low resolution. The Colombian armed conflict has been settling in the last decade, reducing the sense of insecurity in rural areas. People have been moving from cities to the amazonian forest, and we can see their impact on deforestation. Nighttime imagery could allow us to quantify this migration.

# Imports

In [1]:
import rasterio
import numpy as np
import keras.layers.core as core
import keras.layers.convolutional as conv
import keras.models as models
import keras.callbacks
from sklearn.model_selection import KFold
from keras import optimizers
import time

C:\Users\Bertral\venv\cnn_pop\lib\site-packages\h5py\__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


# Parameters

# Preprocessing

In [29]:
def preprocess(filepath, input_tile_size, offset):
    print('opening raster')

    raster = rasterio.open(filepath)

    matrix_x = raster.read(1)
    matrix_y = raster.read(2)

    X = []
    Y = []
    col = 0
    while col + input_tile_size < matrix_x.shape[1]:
        row = 0
        while row + input_tile_size < matrix_x.shape[0]:
            pop = np.sum(matrix_y[row: row + input_tile_size, col: col + input_tile_size])
            # only use tiles that have people living on it
            if pop > 0:
                X.append(matrix_x[row: row + input_tile_size, col: col + input_tile_size])
                Y.append(pop)

            row += offset
        col += offset

    raster.close()
    matrix_x, matrix_y = None, None  # free some memory
    X, Y = np.array(X), np.array(Y)
    X = np.expand_dims(X, axis=3)  # add the color channel as a new dimension
    print('input shape (observations, obs_width, obs_height, channels) : ' + str(X.shape))
    return X, Y


# The neural network

In [30]:
def init_cnn():
    cnn = models.Sequential()
    cnn.add(conv.Convolution2D(filters=64, kernel_size=kernel_size, activation="relu", padding='same',
                               input_shape=(img_rows, img_cols, img_channel_count)))
    
    cnn.add(conv.AveragePooling2D(strides=(2, 2)))
    
    cnn.add(conv.Convolution2D(filters=128, kernel_size=kernel_size, activation="relu", padding='same'))

    cnn.add(conv.AveragePooling2D(strides=(2, 2)))

    cnn.add(conv.Convolution2D(filters=256, kernel_size=kernel_size, activation="relu", padding='same'))
    cnn.add(conv.AveragePooling2D(strides=(2, 2)))
    
    cnn.add(core.Flatten())
    cnn.add(core.Dropout(0.5))
    cnn.add(core.Dense(128))
    cnn.add(core.Dense(1))

    cnn.summary()
    cnn.compile(loss="mean_squared_error", optimizer=optimizers.Adam(lr=0.02, decay=0.0), metrics=["mse", "mae"])
    return cnn

# CALLBACKS
# reduce learning rate when we stopped learning anything
rlrp = keras.callbacks.ReduceLROnPlateau(monitor='loss', factor=0.5, patience=20, verbose=1, mode='auto', min_lr=0.0000001)

# stop learning early if we stopped leaning anything for a longer time
early_stopping = keras.callbacks.EarlyStopping(monitor='loss', min_delta=0.0001, patience=200, verbose=1, mode='auto')

# K-Fold validation

In [31]:
model_birthday = time.strftime("%Y-%m-%d_%H-%M-%S", time.gmtime())  # used to identify generated files (logs and models)
nb_epoch = 2000

# CALLBACKS
# logs for tensorboard
tensorboard = keras.callbacks.TensorBoard(log_dir="logs/" + model_birthday)

# checkpoints
checkpoint = keras.callbacks.ModelCheckpoint('models/' + model_birthday + '.h5', save_weights_only=False)

print('preprocessing')
X, Y = preprocess('../../data/lightpop_merged/adj_2015_brazil.tif', 32, 32)

print('configuring cnn')

# kernel size for each convolution layer
kernel_size = (3, 3)

# input dimensions
img_count, img_rows, img_cols, img_channel_count = X.shape

# k-fold split
kfold = KFold(n_splits = 4, shuffle=True, random_state=None)

# initialize statistics aggregate
kfold_mse = []
kfold_mae = []
kfold_sae = []

print('logs will be saved to logs/' + model_birthday)

for train, test in kfold.split(X, Y):
    
    cnn = init_cnn()

    print('training ...')

    cnn.fit(X[train], Y[train], batch_size=1024, epochs=nb_epoch, verbose=2, callbacks=[tensorboard, rlrp, early_stopping],
            sample_weight=None)

    cnn.save('models/' + model_birthday + '.h5')

    print('model saved to models/' + model_birthday + '.h5')
    
    evaluation = cnn.evaluate(X[test], Y[test], verbose=2, batch_size=1024)
    evaluation = dict(zip(cnn.metrics_names, evaluation))
    kfold_mse.append(evaluation['mean_squared_error'])
    kfold_mae.append(evaluation['mean_absolute_error'])
    kfold_sae.append(evaluation['mean_absolute_error'] * len(Y[test]))
    
    print('K-fold validation results :')
    print('Mean squared error : %.2f (std %.2f)' % (np.mean(kfold_mse), np.std(kfold_mse)))
    print('Mean absolute error : %.2f (std %.2f)' % (np.mean(kfold_mae), np.std(kfold_mae)))
    print('Sum of absolute errors : %.2f (std %.2f)' % (np.mean(kfold_sae), np.std(kfold_sae)))

print('done !')


preprocessing
opening raster
input shape (observations, obs_width, obs_height, channels) : (626684, 32, 32, 1)
configuring cnn
logs will be saved to logs/2018-07-08_19-55-56
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_71 (Conv2D)           (None, 32, 32, 64)        640       
_________________________________________________________________
average_pooling2d_59 (Averag (None, 16, 16, 64)        0         
_________________________________________________________________
conv2d_72 (Conv2D)           (None, 16, 16, 128)       73856     
_________________________________________________________________
average_pooling2d_60 (Averag (None, 8, 8, 128)         0         
_________________________________________________________________
conv2d_73 (Conv2D)           (None, 8, 8, 256)         295168    
_________________________________________________________________
average_pooling2d_61 (Averag (None

Epoch 57/2000
 - 42s - loss: 129526721.8562 - mean_squared_error: 129526721.8562 - mean_absolute_error: 2070.2466
Epoch 58/2000
 - 42s - loss: 119763766.2553 - mean_squared_error: 119763766.2553 - mean_absolute_error: 2098.9206
Epoch 59/2000
 - 42s - loss: 119615696.2446 - mean_squared_error: 119615696.2446 - mean_absolute_error: 2077.6637
Epoch 60/2000
 - 42s - loss: 125506472.2529 - mean_squared_error: 125506472.2529 - mean_absolute_error: 2077.1413
Epoch 61/2000
 - 42s - loss: 122612491.5367 - mean_squared_error: 122612491.5367 - mean_absolute_error: 2117.4095
Epoch 62/2000
 - 42s - loss: 117033289.3713 - mean_squared_error: 117033289.3713 - mean_absolute_error: 2072.9828
Epoch 63/2000
 - 42s - loss: 129571199.9616 - mean_squared_error: 129571199.9616 - mean_absolute_error: 2114.6842
Epoch 64/2000
 - 42s - loss: 110293666.7416 - mean_squared_error: 110293666.7416 - mean_absolute_error: 2058.7290
Epoch 65/2000
 - 42s - loss: 116712625.7081 - mean_squared_error: 116712625.7081 - mean_

Epoch 130/2000
 - 42s - loss: 70290497.7809 - mean_squared_error: 70290497.7809 - mean_absolute_error: 1888.0046
Epoch 131/2000
 - 42s - loss: 64505028.4106 - mean_squared_error: 64505028.4106 - mean_absolute_error: 1853.8037
Epoch 132/2000
 - 42s - loss: 59575516.9622 - mean_squared_error: 59575516.9622 - mean_absolute_error: 1813.5099
Epoch 133/2000
 - 42s - loss: 61207177.5300 - mean_squared_error: 61207177.5300 - mean_absolute_error: 1809.4365
Epoch 134/2000
 - 42s - loss: 65148676.3240 - mean_squared_error: 65148676.3240 - mean_absolute_error: 1854.5170
Epoch 135/2000
 - 42s - loss: 79674990.6477 - mean_squared_error: 79674990.6477 - mean_absolute_error: 1897.1711
Epoch 136/2000
 - 42s - loss: 72223065.0538 - mean_squared_error: 72223065.0538 - mean_absolute_error: 1971.0804
Epoch 137/2000
 - 42s - loss: 62664699.0676 - mean_squared_error: 62664699.0676 - mean_absolute_error: 1889.9652
Epoch 138/2000
 - 42s - loss: 58481649.0861 - mean_squared_error: 58481649.0861 - mean_absolute_

Epoch 202/2000
 - 41s - loss: 34324079.1052 - mean_squared_error: 34324079.1052 - mean_absolute_error: 1542.6108
Epoch 203/2000
 - 41s - loss: 36082406.2713 - mean_squared_error: 36082406.2713 - mean_absolute_error: 1549.0418
Epoch 204/2000
 - 41s - loss: 32773674.0257 - mean_squared_error: 32773674.0257 - mean_absolute_error: 1521.2117
Epoch 205/2000
 - 41s - loss: 38516605.5405 - mean_squared_error: 38516605.5405 - mean_absolute_error: 1567.2291
Epoch 206/2000
 - 41s - loss: 37071836.6786 - mean_squared_error: 37071836.6786 - mean_absolute_error: 1555.7606
Epoch 207/2000
 - 41s - loss: 33008583.2840 - mean_squared_error: 33008583.2840 - mean_absolute_error: 1521.3929
Epoch 208/2000
 - 41s - loss: 34646593.4175 - mean_squared_error: 34646593.4175 - mean_absolute_error: 1526.0526
Epoch 209/2000
 - 41s - loss: 37218560.0671 - mean_squared_error: 37218560.0671 - mean_absolute_error: 1537.8283

Epoch 00209: ReduceLROnPlateau reducing learning rate to 0.004999999888241291.
Epoch 210/2000
 

 - 43s - loss: 22585043.9625 - mean_squared_error: 22585043.9625 - mean_absolute_error: 1354.3558
Epoch 274/2000
 - 43s - loss: 22488575.1456 - mean_squared_error: 22488575.1456 - mean_absolute_error: 1351.5461
Epoch 275/2000
 - 43s - loss: 21355877.0353 - mean_squared_error: 21355877.0353 - mean_absolute_error: 1341.7613
Epoch 276/2000
 - 43s - loss: 22617681.7011 - mean_squared_error: 22617681.7011 - mean_absolute_error: 1350.0131
Epoch 277/2000
 - 43s - loss: 20805750.6689 - mean_squared_error: 20805750.6689 - mean_absolute_error: 1339.9826
Epoch 278/2000
 - 43s - loss: 22564381.2303 - mean_squared_error: 22564381.2303 - mean_absolute_error: 1351.3123
Epoch 279/2000
 - 43s - loss: 22220896.3559 - mean_squared_error: 22220896.3559 - mean_absolute_error: 1346.1876
Epoch 280/2000
 - 43s - loss: 22310320.0685 - mean_squared_error: 22310320.0685 - mean_absolute_error: 1355.1108
Epoch 281/2000
 - 43s - loss: 23105502.9376 - mean_squared_error: 23105502.9376 - mean_absolute_error: 1354.185

...

Epoch 560/2000
 - 42s - loss: 18419301.3884 - mean_squared_error: 18419301.3884 - mean_absolute_error: 1266.4679
Epoch 561/2000
 - 42s - loss: 18295747.3564 - mean_squared_error: 18295747.3564 - mean_absolute_error: 1266.3496
Epoch 562/2000
 - 42s - loss: 18243782.2458 - mean_squared_error: 18243782.2458 - mean_absolute_error: 1264.1084
Epoch 563/2000
 - 42s - loss: 18341479.5272 - mean_squared_error: 18341479.5272 - mean_absolute_error: 1262.0077
Epoch 564/2000
 - 42s - loss: 18580495.5068 - mean_squared_error: 18580495.5068 - mean_absolute_error: 1266.4773
Epoch 565/2000
 - 42s - loss: 17741102.6781 - mean_squared_error: 17741102.6781 - mean_absolute_error: 1262.0467
Epoch 566/2000
 - 42s - loss: 18598515.4355 - mean_squared_error: 18598515.4355 - mean_absolute_error: 1269.2957
Epoch 567/2000
 - 42s - loss: 18401927.6309 - mean_squared_error: 18401927.6309 - mean_absolute_error: 1268.0796
Epoch 568/2000
 - 42s - loss: 18028734.6779 - mean_squared_error: 18028734.6779 - mean_absolute_

Epoch 630/2000
 - 42s - loss: 18091121.8192 - mean_squared_error: 18091121.8192 - mean_absolute_error: 1265.3744
Epoch 631/2000
 - 42s - loss: 18771301.4018 - mean_squared_error: 18771301.4018 - mean_absolute_error: 1266.3807
Epoch 632/2000
 - 42s - loss: 17748357.1816 - mean_squared_error: 17748357.1816 - mean_absolute_error: 1260.1783
Epoch 633/2000
 - 42s - loss: 18184154.1502 - mean_squared_error: 18184154.1502 - mean_absolute_error: 1266.1446
Epoch 634/2000
 - 42s - loss: 18866824.2254 - mean_squared_error: 18866824.2254 - mean_absolute_error: 1269.8507
Epoch 635/2000
 - 42s - loss: 18528052.0007 - mean_squared_error: 18528052.0007 - mean_absolute_error: 1268.4027
Epoch 636/2000
 - 42s - loss: 18049076.0156 - mean_squared_error: 18049076.0156 - mean_absolute_error: 1263.8259
Epoch 637/2000
 - 42s - loss: 18656880.6116 - mean_squared_error: 18656880.6116 - mean_absolute_error: 1269.0835
Epoch 638/2000
 - 42s - loss: 18821520.8574 - mean_squared_error: 18821520.8574 - mean_absolute_

Epoch 701/2000
 - 42s - loss: 18245321.4883 - mean_squared_error: 18245321.4883 - mean_absolute_error: 1263.6835
Epoch 702/2000
 - 42s - loss: 18225448.6275 - mean_squared_error: 18225448.6275 - mean_absolute_error: 1265.3906
Epoch 703/2000
 - 42s - loss: 18632319.6703 - mean_squared_error: 18632319.6703 - mean_absolute_error: 1270.8315
Epoch 704/2000
 - 42s - loss: 17881849.0836 - mean_squared_error: 17881849.0836 - mean_absolute_error: 1262.3193
Epoch 705/2000
 - 42s - loss: 17914359.9287 - mean_squared_error: 17914359.9287 - mean_absolute_error: 1264.9977
Epoch 706/2000
 - 42s - loss: 18201571.2581 - mean_squared_error: 18201571.2581 - mean_absolute_error: 1265.5589
Epoch 707/2000
 - 42s - loss: 19074654.6898 - mean_squared_error: 19074654.6898 - mean_absolute_error: 1273.9677
Epoch 708/2000
 - 42s - loss: 18780876.4548 - mean_squared_error: 18780876.4548 - mean_absolute_error: 1271.1672

Epoch 00708: ReduceLROnPlateau reducing learning rate to 1.5258788721439487e-07.
Epoch 709/2000

Epoch 7/2000
 - 42s - loss: 338416445.1521 - mean_squared_error: 338416445.1521 - mean_absolute_error: 2921.3378
Epoch 8/2000
 - 42s - loss: 281446796.8244 - mean_squared_error: 281446796.8244 - mean_absolute_error: 2818.5610
Epoch 9/2000
 - 42s - loss: 327288766.4734 - mean_squared_error: 327288766.4734 - mean_absolute_error: 2916.2389
Epoch 10/2000
 - 42s - loss: 380525713.5936 - mean_squared_error: 380525713.5936 - mean_absolute_error: 3050.6443
Epoch 11/2000
 - 42s - loss: 314730427.1533 - mean_squared_error: 314730427.1533 - mean_absolute_error: 2883.3593
Epoch 12/2000
 - 42s - loss: 297865063.5342 - mean_squared_error: 297865063.5342 - mean_absolute_error: 2845.7997
Epoch 13/2000
 - 42s - loss: 278796378.5106 - mean_squared_error: 278796378.5106 - mean_absolute_error: 2702.7251
Epoch 14/2000
 - 42s - loss: 316716365.5005 - mean_squared_error: 316716365.5005 - mean_absolute_error: 2750.9926
Epoch 15/2000
 - 42s - loss: 291435360.4067 - mean_squared_error: 291435360.4067 - mean_abs

KeyboardInterrupt: 

# Training without validation

In [35]:
model_birthday = time.strftime("%Y-%m-%d_%H-%M-%S", time.gmtime())  # used to identify generated files (logs and models)
nb_epoch = 1000

# CALLBACKS
# logs for tensorboard
tensorboard = keras.callbacks.TensorBoard(log_dir="logs/" + model_birthday)

# checkpoints
checkpoint = keras.callbacks.ModelCheckpoint('models/' + model_birthday + '.h5', save_weights_only=False)

print('preprocessing')

preprocess('../../data/lightpop_merged/adj_2015_safrica_namibia.tif', 32, 8)

print('configuring cnn')

# kernel size for each convolution layer
kernel_size = (3, 3)

# input dimensions
img_count, img_rows, img_cols, img_channel_count = X.shape

print('logs will be saved to logs/' + model_birthday)
    
cnn = init_cnn()

print('training ...')

cnn.fit(X, Y, batch_size=1024, epochs=nb_epoch, verbose=2,
        callbacks=[tensorboard, checkpoint, rlrp, early_stopping], sample_weight=None)

cnn.save('models/' + model_birthday + '.h5')

print('model saved to models/' + model_birthday + '.h5')

print('done !')

preprocessing
opening raster
input shape (observations, obs_width, obs_height, channels) : (173448, 32, 32, 1)
configuring cnn
logs will be saved to logs/2018-07-09_05-56-23
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_77 (Conv2D)           (None, 32, 32, 64)        640       
_________________________________________________________________
average_pooling2d_65 (Averag (None, 16, 16, 64)        0         
_________________________________________________________________
conv2d_78 (Conv2D)           (None, 16, 16, 128)       73856     
_________________________________________________________________
average_pooling2d_66 (Averag (None, 8, 8, 128)         0         
_________________________________________________________________
conv2d_79 (Conv2D)           (None, 8, 8, 256)         295168    
_________________________________________________________________
average_pooling2d_67 (Averag (None

Epoch 57/1000
 - 55s - loss: 105912767.4040 - mean_squared_error: 105912767.4040 - mean_absolute_error: 1812.9724
Epoch 58/1000
 - 55s - loss: 75990604.3504 - mean_squared_error: 75990604.3504 - mean_absolute_error: 1719.4082
Epoch 59/1000
 - 55s - loss: 92283209.0347 - mean_squared_error: 92283209.0347 - mean_absolute_error: 1792.3590
Epoch 60/1000
 - 55s - loss: 92277300.8406 - mean_squared_error: 92277300.8406 - mean_absolute_error: 1765.3903
Epoch 61/1000
 - 55s - loss: 108070485.3749 - mean_squared_error: 108070485.3749 - mean_absolute_error: 1847.0708
Epoch 62/1000
 - 55s - loss: 71361262.7001 - mean_squared_error: 71361262.7001 - mean_absolute_error: 1683.4589
Epoch 63/1000
 - 55s - loss: 93693732.5650 - mean_squared_error: 93693732.5650 - mean_absolute_error: 1785.1089
Epoch 64/1000
 - 55s - loss: 80076930.8192 - mean_squared_error: 80076930.8192 - mean_absolute_error: 1715.4822
Epoch 65/1000
 - 55s - loss: 84198864.4473 - mean_squared_error: 84198864.4473 - mean_absolute_error

Epoch 130/1000
 - 56s - loss: 47703341.4540 - mean_squared_error: 47703341.4540 - mean_absolute_error: 1552.9559
Epoch 131/1000
 - 56s - loss: 65947311.7207 - mean_squared_error: 65947311.7207 - mean_absolute_error: 1660.6659
Epoch 132/1000
 - 56s - loss: 78795391.1557 - mean_squared_error: 78795391.1557 - mean_absolute_error: 1713.8198
Epoch 133/1000
 - 56s - loss: 51887227.2032 - mean_squared_error: 51887227.2032 - mean_absolute_error: 1557.4537
Epoch 134/1000
 - 56s - loss: 55788212.4212 - mean_squared_error: 55788212.4212 - mean_absolute_error: 1574.8198
Epoch 135/1000
 - 56s - loss: 49501956.7312 - mean_squared_error: 49501956.7312 - mean_absolute_error: 1540.4076
Epoch 136/1000
 - 56s - loss: 53287244.9637 - mean_squared_error: 53287244.9637 - mean_absolute_error: 1571.8760
Epoch 137/1000
 - 56s - loss: 53159235.2148 - mean_squared_error: 53159235.2148 - mean_absolute_error: 1570.4900
Epoch 138/1000
 - 55s - loss: 62464302.9063 - mean_squared_error: 62464302.9063 - mean_absolute_

 - 55s - loss: 22003572.4400 - mean_squared_error: 22003572.4400 - mean_absolute_error: 1242.0754
Epoch 202/1000
 - 54s - loss: 21222401.3155 - mean_squared_error: 21222401.3155 - mean_absolute_error: 1226.9420
Epoch 203/1000
 - 54s - loss: 19512010.4705 - mean_squared_error: 19512010.4705 - mean_absolute_error: 1217.5726
Epoch 204/1000
 - 54s - loss: 21784288.3758 - mean_squared_error: 21784288.3758 - mean_absolute_error: 1228.8345
Epoch 205/1000
 - 54s - loss: 21277887.0743 - mean_squared_error: 21277887.0743 - mean_absolute_error: 1227.4031
Epoch 206/1000
 - 54s - loss: 21171677.3713 - mean_squared_error: 21171677.3713 - mean_absolute_error: 1225.7191
Epoch 207/1000
 - 54s - loss: 18703315.6112 - mean_squared_error: 18703315.6112 - mean_absolute_error: 1207.3929
Epoch 208/1000
 - 54s - loss: 19439726.8809 - mean_squared_error: 19439726.8809 - mean_absolute_error: 1214.2312
Epoch 209/1000
 - 54s - loss: 21859927.9229 - mean_squared_error: 21859927.9229 - mean_absolute_error: 1229.069

Epoch 274/1000
 - 59s - loss: 18861725.9839 - mean_squared_error: 18861725.9839 - mean_absolute_error: 1173.4917
Epoch 275/1000
 - 59s - loss: 16508538.9096 - mean_squared_error: 16508538.9096 - mean_absolute_error: 1147.2165
Epoch 276/1000
 - 59s - loss: 17890048.6820 - mean_squared_error: 17890048.6820 - mean_absolute_error: 1166.7013
Epoch 277/1000
 - 58s - loss: 19230648.4783 - mean_squared_error: 19230648.4783 - mean_absolute_error: 1172.8169
Epoch 278/1000
 - 58s - loss: 17151198.8763 - mean_squared_error: 17151198.8763 - mean_absolute_error: 1158.7306
Epoch 279/1000
 - 58s - loss: 17257244.5480 - mean_squared_error: 17257244.5480 - mean_absolute_error: 1156.7554
Epoch 280/1000
 - 58s - loss: 16863306.0671 - mean_squared_error: 16863306.0671 - mean_absolute_error: 1158.8955
Epoch 281/1000
 - 58s - loss: 18774696.6200 - mean_squared_error: 18774696.6200 - mean_absolute_error: 1174.6477
Epoch 282/1000
 - 58s - loss: 16671795.4234 - mean_squared_error: 16671795.4234 - mean_absolute_

...

 - 55s - loss: 11179278.3179 - mean_squared_error: 11179278.3179 - mean_absolute_error: 1047.0597
Epoch 490/1000
 - 55s - loss: 11629384.4817 - mean_squared_error: 11629384.4817 - mean_absolute_error: 1054.4711
Epoch 491/1000
 - 55s - loss: 11636847.2730 - mean_squared_error: 11636847.2730 - mean_absolute_error: 1052.2104
Epoch 492/1000
 - 56s - loss: 11126764.3069 - mean_squared_error: 11126764.3069 - mean_absolute_error: 1049.0556
Epoch 493/1000
 - 56s - loss: 11567564.1731 - mean_squared_error: 11567564.1731 - mean_absolute_error: 1052.0754
Epoch 494/1000
 - 56s - loss: 10874206.0366 - mean_squared_error: 10874206.0366 - mean_absolute_error: 1046.3293
Epoch 495/1000
 - 55s - loss: 11676647.1266 - mean_squared_error: 11676647.1266 - mean_absolute_error: 1052.3933
Epoch 496/1000
 - 55s - loss: 12038469.5352 - mean_squared_error: 12038469.5352 - mean_absolute_error: 1055.7134
Epoch 497/1000
 - 55s - loss: 11490476.8219 - mean_squared_error: 11490476.8219 - mean_absolute_error: 1052.041

Epoch 561/1000
 - 54s - loss: 11544935.7502 - mean_squared_error: 11544935.7502 - mean_absolute_error: 1052.1847
Epoch 562/1000
 - 54s - loss: 11443493.6621 - mean_squared_error: 11443493.6621 - mean_absolute_error: 1046.6036
Epoch 563/1000
 - 55s - loss: 11790810.5822 - mean_squared_error: 11790810.5822 - mean_absolute_error: 1049.2401
Epoch 564/1000
 - 57s - loss: 11452566.7097 - mean_squared_error: 11452566.7097 - mean_absolute_error: 1048.4027
Epoch 565/1000
 - 57s - loss: 11385689.3357 - mean_squared_error: 11385689.3357 - mean_absolute_error: 1048.8186
Epoch 566/1000
 - 57s - loss: 11359764.4633 - mean_squared_error: 11359764.4633 - mean_absolute_error: 1044.4574
Epoch 567/1000
 - 57s - loss: 11425219.7351 - mean_squared_error: 11425219.7351 - mean_absolute_error: 1048.4491
Epoch 568/1000
 - 57s - loss: 11291015.3655 - mean_squared_error: 11291015.3655 - mean_absolute_error: 1044.9176
Epoch 569/1000
 - 56s - loss: 11374023.0653 - mean_squared_error: 11374023.0653 - mean_absolute_

Epoch 632/1000
 - 56s - loss: 10648800.1562 - mean_squared_error: 10648800.1562 - mean_absolute_error: 1041.3274
Epoch 633/1000
 - 56s - loss: 11032266.4054 - mean_squared_error: 11032266.4054 - mean_absolute_error: 1044.3641
Epoch 634/1000
 - 56s - loss: 11205940.6779 - mean_squared_error: 11205940.6779 - mean_absolute_error: 1044.2443
Epoch 635/1000
 - 56s - loss: 11186825.3650 - mean_squared_error: 11186825.3650 - mean_absolute_error: 1048.6386
Epoch 636/1000
 - 56s - loss: 11422577.9576 - mean_squared_error: 11422577.9576 - mean_absolute_error: 1048.2638
Epoch 637/1000
 - 56s - loss: 11391069.1955 - mean_squared_error: 11391069.1955 - mean_absolute_error: 1046.3227
Epoch 638/1000
 - 56s - loss: 11295600.3492 - mean_squared_error: 11295600.3492 - mean_absolute_error: 1045.9745
Epoch 639/1000
 - 56s - loss: 10966781.7539 - mean_squared_error: 10966781.7539 - mean_absolute_error: 1045.6222
Epoch 640/1000
 - 56s - loss: 11019455.2883 - mean_squared_error: 11019455.2883 - mean_absolute_

Epoch 703/1000
 - 57s - loss: 11232449.4762 - mean_squared_error: 11232449.4762 - mean_absolute_error: 1044.8986
Epoch 704/1000
 - 57s - loss: 11492815.5362 - mean_squared_error: 11492815.5362 - mean_absolute_error: 1045.4617
Epoch 705/1000
 - 56s - loss: 11364833.7875 - mean_squared_error: 11364833.7875 - mean_absolute_error: 1047.3381
Epoch 706/1000
 - 56s - loss: 11298572.6525 - mean_squared_error: 11298572.6525 - mean_absolute_error: 1048.4420
Epoch 707/1000
 - 56s - loss: 11324975.5862 - mean_squared_error: 11324975.5862 - mean_absolute_error: 1046.8128
Epoch 708/1000
 - 57s - loss: 11519446.6166 - mean_squared_error: 11519446.6166 - mean_absolute_error: 1047.6149
Epoch 709/1000
 - 57s - loss: 11115370.5582 - mean_squared_error: 11115370.5582 - mean_absolute_error: 1045.0479
Epoch 710/1000
 - 57s - loss: 11205608.6976 - mean_squared_error: 11205608.6976 - mean_absolute_error: 1046.1316
Epoch 711/1000
 - 56s - loss: 11646328.8112 - mean_squared_error: 11646328.8112 - mean_absolute_

 - 56s - loss: 11148455.9185 - mean_squared_error: 11148455.9185 - mean_absolute_error: 1046.1651
Epoch 774/1000
 - 56s - loss: 11007552.0633 - mean_squared_error: 11007552.0633 - mean_absolute_error: 1044.6005
Epoch 775/1000
 - 56s - loss: 11095307.5215 - mean_squared_error: 11095307.5215 - mean_absolute_error: 1045.5458
Epoch 776/1000
 - 57s - loss: 11136725.9092 - mean_squared_error: 11136725.9092 - mean_absolute_error: 1043.5933
Epoch 777/1000
 - 56s - loss: 11100341.7523 - mean_squared_error: 11100341.7523 - mean_absolute_error: 1046.2521
Epoch 778/1000
 - 56s - loss: 10953730.2249 - mean_squared_error: 10953730.2249 - mean_absolute_error: 1043.5623
Epoch 779/1000
 - 56s - loss: 11221346.6188 - mean_squared_error: 11221346.6188 - mean_absolute_error: 1045.8692
Epoch 780/1000
 - 56s - loss: 11081886.3886 - mean_squared_error: 11081886.3886 - mean_absolute_error: 1045.8045
Epoch 781/1000
 - 57s - loss: 11577963.2888 - mean_squared_error: 11577963.2888 - mean_absolute_error: 1047.055

# Prediction

In [36]:
prediction_dataset = '../../data/lightrasters_noaa/2012_south_america.tif'
input_tile_size = 32

print('loading model')

cnn = models.load_model("models/" + model_birthday + ".h5")

print('opening raster')

raster = rasterio.open(prediction_dataset)
band = raster.read(1)
profile = raster.profile
profile.update(count=1)
width, height = raster.width, raster.height

# preprocess
matrix_x = raster.read(1)
tiles_x = []
y = 0
while y + input_tile_size < matrix_x.shape[1]:
    x = 0
    while x + input_tile_size < matrix_x.shape[0]:
        tiles_x.append(matrix_x[x: x + input_tile_size, y: y + input_tile_size])
        x += input_tile_size
    y += input_tile_size
testX = np.array(tiles_x)
raster.close()
matrix_x = None
tiles_x = None
testX = np.expand_dims(testX, axis=3)

print('generating raster')

predicted_tiles = cnn.predict(testX, verbose=0)

predicted_raster = np.zeros(shape=(raster.height, raster.width))
y = 0
pred_index = 0
while y + input_tile_size < width:
    x = 0
    while x + input_tile_size < height:
        in_tile = band[x: x + input_tile_size, y: y + input_tile_size]
        if np.max(in_tile) <= 0:
            predicted_raster[x: x + input_tile_size, y: y + input_tile_size] = 0
        else:
            weights = in_tile / np.max(in_tile)  # normalize visible light between 0 and 1 to avoid overflows
            weights = np.exp(weights) - 1  # visible light is perceived logarithmically => counteract with exp
            weights = weights / np.sum(weights)  # the sum of all weights must be 1
            predicted_raster[x: x + input_tile_size, y: y + input_tile_size] = predicted_tiles[pred_index] * weights

        pred_index += 1
        x += input_tile_size
    y += input_tile_size

predicted_raster = np.array(predicted_raster)

with rasterio.open('predictions/' + model_birthday + '.tif', 'w', **profile) as dst:
    dst.write(predicted_raster.astype(rasterio.float32), 1)
print("prediction saved to predictions/" + model_birthday + '-2018.tif')

print('prediction done !')


loading model
opening raster
generating raster
prediction saved to predictions/2018-07-09_05-56-23.tif
prediction done !
